In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.classify.maxent import MaxentClassifier
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import gc

from catboost import CatBoostClassifier

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv('../data/train_data.csv')
df['type'] = df['id'].str[:10]
df = df[df['type'] == 'sodepami_v']
df = df.sample(688000)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
# df1 = pd.read_csv('../input/train-sim/train.csv')
# df1 = df1[df1['price_vnd']>=5e6]

In [ ]:
# df = pd.concat([df,df1])

In [ ]:
df.shape

In [ ]:
def removing_wrong_phone_number(df):
    df['phone_number'] = df['phone_number'].astype(str)
    list_number = df['phone_number'].to_list()
    list_len = []

    for i in range(len(list_number)):
        list_len.append(len(list_number[i]))

    df['length'] = list_len

    df = df[df['length'] != 11]

    return df

In [ ]:
df = removing_wrong_phone_number(df)

In [ ]:
def creating_10_feats_from_single_pos(df):
    list_number = df['phone_number'].to_list()
    list_char = np.full((df.shape[0], 9), "-1")

    for i in range(len(list_number)):
        k = 8
        for j in range(len(list_number[i])-1, 0, -1):
            list_char[i][k] = list_number[i][j]
            k-=1
    
            if(k==0):
                list_char[i][k] = list_number[i][0:j]
                break
    for i in range(9):
        df['pos_' + str(i+1)] = list_char[:,i]
    return df

In [ ]:
df = creating_10_feats_from_single_pos(df)
df.head(10)

In [ ]:
def features_from_counting_numbers(df):
    list_number = df['phone_number'].to_list()
    
    list_count = np.full((df.shape[0], 10), 0)
    
    for i in range(len(list_number)):
        for j in range(len(list_number[i])):
            tmp = int(list_number[i][j])
            list_count[i][tmp] += 1  
    for i in range(10):
        df['count_' + str(i)] = list_count[:,i]
    
    max_count = [0 for i in range(len(list_number))]
    for i in range(len(list_number)):
        max_c = 0
        pos = 0
        for j in range(len(list_count[i])):
            if list_count[i][j] >= max_c:
                max_c = list_count[i][j]
                pos = j
        #print(max_c)
        max_count[i] = pos  
        
    df['max_count'] = max_count
    return df

In [ ]:
df = features_from_counting_numbers(df)

In [ ]:
def combo_feat(df):
    list_number = df['phone_number'].tolist()

    list_2_last = list() #extract 2 last digit

    list_3_last = list() #extract 3 last digit

    list_4_last = list() #extract 4 last digit

    list_5_last = list() #extract 5 last digit

    list_first = list() #extract first digit after 0

    list_combo_3_identical_bool = list() # the phone number contain 111, 222, ....999 or not

    list_combo_3_identical_cat = list() # the phone number contain which one of 111, 222, ....999

    list_combo_4_identical_bool = list() # the phone number contain 1111, 2222, ... or not

    list_combo_4_identical_cat = list()

    list_combo_5_identical_bool = list() # the phone number contain 11111, 22222, ... or not

    list_combo_5_identical_cat = list()

    list_combo_6_identical_bool = list() 

    list_combo_6_identical_cat = list() 

    list_3_combo_s = list() # tam hoa <5

    list_3_combo_57 = list() # tam hoa 5,7

    list_3_combo_689 = list() # tam hoa 6,8,9

    list_4_combo_s = list() # tu quy <5

    list_4_combo_57 = list() # tu quy 5,7

    list_4_combo_689 = list() # tu quy 6,8,9


    for i in range(len(list_number)):
            str_tmp_2_last = list_number[i][-2:]
            list_2_last.append(str_tmp_2_last)

            str_tmp_3_last = list_number[i][-3:]
            list_3_last.append(str_tmp_3_last)

            str_tmp_4_last = list_number[i][-4:]
            list_4_last.append(str_tmp_4_last)

            str_tmp_5_last = list_number[i][-5:]
            list_5_last.append(str_tmp_5_last)

            str_tmp_first = list_number[i][0]
            list_first.append(str_tmp_first)

            list_combo_3 = {'000','111','222','333','444','555','666','777','888','999'}
            combo_3_score = 0
            combo_3_cat = ""

            combo_3_s = 0
            combo_3_57 = 0
            combo_3_689 = 0

            for str_tmp in list_combo_3:
                if str_tmp in list_number[i]:
                    combo_3_score = combo_3_score + 1
                    combo_3_cat += str_tmp

                    x = (int)(str_tmp[0])

                    if(x in {0,1,2,3,4} ): 
                        combo_3_s = combo_3_s + 1
                    elif (x == 5 or x == 7 ):
                        combo_3_57 = combo_3_57 + 1    
                    else:
                        combo_3_689 = combo_3_689 + 1

            list_combo_3_identical_bool.append(combo_3_score)
            list_combo_3_identical_cat.append(combo_3_cat)

            list_3_combo_s.append(combo_3_s)
            list_3_combo_57.append(combo_3_57)
            list_3_combo_689.append(combo_3_689)

            list_combo_4 = {'0000','1111','2222','3333','4444','5555','6666','7777','8888','9999'}
            combo_4_score = 0
            combo_4_cat = ""

            combo_4_s = 0
            combo_4_57 = 0
            combo_4_689 = 0
            for str_tmp in list_combo_4:
                if str_tmp in list_number[i]:
                    combo_4_score = combo_4_score + 1
                    combo_4_cat += str_tmp

                    x = (int)(str_tmp[0])

                    if(x in {0,1,2,3,4} ): 
                        combo_4_s = combo_4_s + 1
                    elif (x == 5 or x == 7 ):
                        combo_4_57 = combo_4_57 + 1    
                    else:
                        combo_4_689 = combo_4_689 + 1            


            list_combo_4_identical_bool.append(combo_4_score)
            list_combo_4_identical_cat.append(combo_4_cat)

            list_4_combo_s.append(combo_4_s)
            list_4_combo_57.append(combo_4_57)
            list_4_combo_689.append(combo_4_689)

            list_combo_5 = {'00000','11111','22222','33333','44444','55555','66666','77777','88888','99999'}
            combo_5_score = 0
            combo_5_cat = ""
            for str_tmp in list_combo_5:
                if str_tmp in list_number[i]:
                    combo_5_score = combo_5_score + 1
                    combo_5_cat += str_tmp

            list_combo_5_identical_bool.append(combo_5_score)
            list_combo_5_identical_cat.append(combo_5_cat)


            list_combo_6 = {'000000','111111','222222','333333','444444','555555','666666','777777','888888','999999'}
            combo_6_score = 0
            combo_6_cat = ""

            for str_tmp in list_combo_6:
                if str_tmp in list_number[i]:
                    combo_6_score = combo_6_score + 1
                    combo_6_cat += str_tmp

            list_combo_6_identical_bool.append(combo_6_score)
            list_combo_6_identical_cat.append(combo_6_cat)

    df['2_last'] = list_2_last
    df['3_last'] = list_3_last
    df['4_last'] = list_4_last
    df['5_last'] = list_5_last
    df['first'] = list_first

    df['3_combo_bool'] = list_combo_3_identical_bool
    df['3_combo_cat'] = list_combo_3_identical_cat

    df['4_combo_bool'] = list_combo_4_identical_bool
    df['4_combo_cat'] = list_combo_4_identical_cat

    df['5_combo_bool'] = list_combo_5_identical_bool
    df['5_combo_cat'] = list_combo_5_identical_cat

    df['6_combo_bool'] = list_combo_6_identical_bool
    df['6_combo_cat'] = list_combo_6_identical_cat

    df['3_combo_s'] = list_3_combo_s
    df['3_combo_57'] = list_3_combo_57
    df['3_combo_689'] = list_3_combo_689

    df['4_combo_s'] = list_4_combo_s
    df['4_combo_57'] = list_4_combo_57
    df['4_combo_689'] = list_4_combo_689
        
    return df

In [ ]:
df = combo_feat(df)

In [ ]:
df[df['3_combo_bool']!=0]

In [ ]:
# def lucky_2_last(df):
#     list_2_lucky_last_bool = list()
#     list_2_lucky_last_cat = list()

#     for i in list_2_last:
#         if (i == '68' or i == '86' or i == '78' or i == '79' or i == '39' or i == '38' or i =='83'):
#             list_2_lucky_last_bool.append(1)
#             list_2_lucky_last_cat.append(i)
#         else:
#             list_2_lucky_last_bool.append(0)
#             list_2_lucky_last_cat.append('')


#     df['2_lucky_last_bool'] = list_2_lucky_last_bool # check if the phone number has 2 last "lucky number" or not
#     df['2_lucky_last_cat'] = list_2_lucky_last_cat
    
#     return df


In [ ]:
def sim_nam_sinh(df):
    list_nam_sinh_bool = []
    list_number = df['phone_number'].to_list()
    list_nam_sinh = []
    for i in list_number:
        if 1900 <= int(i[-4:]) <= 2022:
            list_nam_sinh.append(int(i[-4:]))
            list_nam_sinh_bool.append(1)
        else:
            list_nam_sinh.append(0)
            list_nam_sinh_bool.append(0)
            
    
    df['nam_sinh'] = list_nam_sinh
    df['nam_sinh_bool'] = list_nam_sinh_bool
    df['nam_sinh'] = df['nam_sinh'].astype(str)
    return df

In [ ]:
df = sim_nam_sinh(df)

In [ ]:
# df['nam_sinh'][df['nam_sinh_bool']!=0]

In [ ]:
def symetric_4length_and_samesubtring_features(df):
    list_number = df['phone_number'].to_list()
    list_length_palindrome = []
    list_bool_palindrome = []
    list_palindrome = []
    list_same_substr = []
    list_same_substr_length = []
    list_bool_same_substr = []
    for i in range(len(list_number)):
        
        res = [list_number[i][x: y]+str(x) for x in range(len(list_number[i])-1)
              for y in range(x + 2, len(list_number[i])+1)]
                
        tmp = ""
        for j in res:
            if len(j[:-1])>=4  and len(j[:-1]) > len(tmp) and j[:-1] == j[:-1][::-1]:
                tmp = j[:-1]
        
         
        list_palindrome.append(tmp)
        list_length_palindrome.append(len(tmp))
        
        
        tmp2 = ""
                
                
        for j in range(len(res)-1):
            
            u1_n = (len(list_number[i]) - 1 - int(res[j][-1])) * len(res[j][:-1])
            ps = int(len(res[j][:-1]) * ( len(res[j][:-1])-1) / 2)
            
            
            for z in range(j + (u1_n - ps),len(res)):
                if res[j][:-1] == res[z][:-1] and len(res[j][:-1]) > len(tmp2):
                    tmp2 = res[z][:-1]
  
        list_same_substr.append(tmp2)
        list_same_substr_length.append(len(tmp2))
    
    for i in range(len(list_length_palindrome)):
        if list_length_palindrome[i] != 0:
            list_bool_palindrome.append(1)
        else:
            list_bool_palindrome.append(0)
        
        if list_same_substr_length[i] != 0:
            list_bool_same_substr.append(1)
        else:
            list_bool_same_substr.append(0)
        
    df['palindrome_length'] = list_length_palindrome
    df['palindrome'] = list_palindrome
    df['bool_palindrome'] = list_bool_palindrome
    df['list_same'] = list_same_substr
    df['list_same_substr_length'] = list_same_substr_length
    df['bool_substr'] = list_bool_same_substr
    return df

In [ ]:
df = symetric_4length_and_samesubtring_features(df)
df.head(10)

In [ ]:
df[df['bool_substr']==1]

In [ ]:
# def combo_features(df):
#     list_number = df['phone_number'].tolist()
    
#     list_2_last = list() #extract 2 last digit
    
#     list_3_last = list() #extract 3 last digit
    
#     list_4_last = list() #extract 4 last digit
    
#     list_5_last = list() #extract 5 last digit
    
#     list_combo_3_identical = list() # the phone number contain 111, 222, ....999 or not
    
#     list_combo_4_identical = list() # the phone number contain 1111, 2222, ... or not
    
#     list_combo_5_identical = list() # the phone number contain 11111, 22222, ... or not
    
#     list_combo_6_identical = list() # the phone number contain 11111, 22222, ... or not
    
    
    
#     for i in range(len(list_number)):
#         str_2_last = list_number[i][-2:]
#         list_2_last.append(str_2_last)
    
#         str_3_last = list_number[i][-3:]
#         list_3_last.append(str_3_last)
    
#         str_4_last = list_number[i][-4:]
#         list_4_last.append(str_4_last)
    
#         str_5_last = list_number[i][-5:]
#         list_5_last.append(str_5_last)
    
    
#         list_combo_3 = ['000','111','222','333','444','555','666','777','888','999']
    
#         tmp = ""
#         for j in range(len(list_combo_3)):
#             if list_combo_3[j] in list_number[i]:
#                 tmp += list_combo_3[j]
    
#         list_combo_3_identical.append(tmp)
    
#         list_combo_4 = ['0000','1111','2222','3333','4444','5555','6666','7777','8888','9999']
#         tmp_4 = ""
    
#         for j in range(len(list_combo_4)):
#           if list_combo_4[j] in list_number[i]:
#                 tmp_4 += list_combo_4[j]
        
#         list_combo_4_identical.append(tmp_4)
    
#         list_combo_5 = ['00000','11111','22222','33333','44444','55555','66666','77777','88888','99999']
#         tmp_5 = ""
    
#         for j in range(len(list_combo_5)):
#             if list_combo_5[j] in list_number[i]:
#                 tmp_5 += list_combo_5[j]
        
#         list_combo_5_identical.append(tmp_5)
    
    
#         list_combo_6 = ['000000','111111','222222','333333','444444','555555','666666','777777','888888','999999']
#         tmp_6 = ""
    
#         for j in range(len(list_combo_6)):
#             if list_combo_6[j] in list_number[i]:
#                 tmp_6 += list_combo_6[j]
        
#         list_combo_6_identical.append(tmp_6)
    
#     df['2_last'] = list_2_last
#     df['3_last'] = list_3_last
#     df['4_last'] = list_4_last
#     df['5_last'] = list_5_last
#     df['3_combo'] = list_combo_3_identical
#     df['4_combo'] = list_combo_4_identical
#     df['5_combo'] = list_combo_5_identical
#     df['6_combo'] = list_combo_6_identical
    
#     return df

In [ ]:
# df = combo_features(df)
# df.head()

In [ ]:
def count_combo_features(df):
    list_number = df['phone_number'].to_list()
    list_combo_3_count = list() # the phone number contain 111, 222, ....999 or not
    
    list_combo_4_count = list() # the phone number contain 1111, 2222, ... or not
    
    list_combo_5_count = list() # the phone number contain 11111, 22222, ... or not
    
    list_combo_6_count = list() # the phone number contain 11111, 22222, ... or not
    
    
    
    for i in range(len(list_number)):
    
        list_combo_3 = {'000','111','222','333','444','555','666','777','888','999'}
        combo_3_score = 0
    
        for j in list_combo_3:
            if j in list_number[i]:
                combo_3_score = combo_3_score + 1
    
        list_combo_3_count.append(combo_3_score)
    
        list_combo_4 = {'0000','1111','2222','3333','4444','5555','6666','7777','8888','9999'}
        combo_4_score = 0
    
        for j in list_combo_4:
            if j in list_number[i]:
                combo_4_score = combo_4_score + 1
        
        list_combo_4_count.append(combo_4_score)
    
        list_combo_5 = {'00000','11111','22222','33333','44444','55555','66666','77777','88888','99999'}
        combo_5_score = 0
    
        for j in list_combo_5:
            if j in list_number[i]:
                combo_5_score = combo_5_score + 1
        
        list_combo_5_count.append(combo_5_score)
    
    
        list_combo_6 = {'000000','111111','222222','333333','444444','555555','666666','777777','888888','999999'}
        combo_6_score = 0
    
        for j in list_combo_6:
            if j in list_number[i]:
                combo_6_score = combo_6_score + 1
        
        list_combo_6_count.append(combo_6_score)
        
    df['3_combo_count'] = list_combo_3_count
    df['4_combo_count'] = list_combo_4_count
    df['5_combo_count'] = list_combo_5_count
    df['6_combo_count'] = list_combo_6_count
    
    return df

In [ ]:
df = count_combo_features(df)
df.head()

In [ ]:
def features_2_inden_and_2_lucky_last(df):
    list_number = df['phone_number'].to_list()
    
    list_2_iden = list()
    
    list_2_combo = ['00','11','22','33','44','55','66','77','88','99']
    for i in list_number:
        tmp = ""
        for j in list_2_combo:
            if (j in i):
                tmp += j
            else:
                tmp += ""
                
        list_2_iden.append(tmp)
    
    list_2_lucky_last = list()
    
    list_lucky = ['68','86','78','79','39','38','83']
    for i in list_number:
      tmp = ""
      for j in list_lucky:
        if (j in i):
            tmp += j
        else:
            tmp += ""
    
      list_2_lucky_last.append(tmp)
    
    list_bool_2_lucky = []
    for i in list_2_lucky_last:
        if i != "":
            list_bool_2_lucky.append(1)
        else:
            list_bool_2_lucky.append(0)
    
    list_bool_2_iden = []
    for i in list_2_iden:
        if i != "":
            list_bool_2_iden.append(1)
        else:
            list_bool_2_iden.append(0)
    
    df['2_lucky_bool'] = list_bool_2_lucky
    df['2_iden_bool'] = list_bool_2_iden
    df['2_iden'] = list_2_iden
    df['2_lucky'] = list_2_lucky_last
    return df

In [ ]:
df = features_2_inden_and_2_lucky_last(df)
df.head(8)

In [ ]:
df[df['2_iden_bool']==1]

In [ ]:
def last_special_case(df):
    
    list_3_last = df['3_last'].to_list()
    list_4_last = df['4_last'].to_list()
    list_5_last = df['5_last'].to_list()
    
    list_3_last_iden = list()
    
    list_3_last_bool = []
    list_4_last_bool = []
    list_5_last_bool = []
    for x in list_3_last:
        if (x == '000' or x == '111' or x == '222' or x == '333' or x == '444' or x == '555' or x == '666' or x == '777' or x == '888' or x == '999'):
            list_3_last_iden.append(x)
            list_3_last_bool.append(1)
        else:
            list_3_last_iden.append("n")
            list_3_last_bool.append(0)
    
    
    df['3_last_iden'] = list_3_last_iden # if the phone number have 3 last number indentical or not
    df['3_last_iden_bool'] = list_3_last_bool
    
    list_4_last_iden = list()
    
    for x in list_4_last:
        if (x == '0000' or x == '1111' or x == '2222' or x == '3333' or x == '4444' or x == '5555' or x == '6666' or x == '7777' or x == '8888' or x == '9999'):
            list_4_last_iden.append(x)
            list_4_last_bool.append(1)
        else:
            list_4_last_iden.append("n")
            list_4_last_bool.append(0)
    
    
    df['4_last_iden'] = list_4_last_iden # if the phone number have 4 last number indentical or not
    df['4_last_iden_bool'] = list_4_last_bool
    
    list_5_last_iden = list()
    
    for x in list_5_last:
        if (x == '00000' or x == '11111' or x == '22222' or x == '33333' or x == '44444' or x == '55555' or x == '66666' or x == '77777' or x == '88888' or x == '99999'):
            list_5_last_iden.append(x)
            list_5_last_bool.append(1)
        else:
            list_5_last_iden.append("n")
            list_5_last_bool.append(0)
    
    
    df['5_last_iden'] = list_5_last_iden # if the phone number have 4 last number indentical or not
    df['5_last_iden_bool'] = list_5_last_bool
    
    list_3_last_increasing_bool = []
    list_4_last_increasing_bool = []
    list_5_last_increasing_bool = []
    
    list_3_last_increasing_sample = ('012','123','234','345','456','567','678','789','024','135','246','357','468','579','036','147','258','369','048','159')
    list_3_last_increasing = list()
    
    for x in list_3_last:
      check = 0
      for j in list_3_last_increasing_sample:
        if j in x:
            list_3_last_increasing.append(x)
            check = 1
            list_3_last_increasing_bool.append(1)
            break
      if check == 0:
        list_3_last_increasing.append("n") # 3-last increasing number
        list_3_last_increasing_bool.append(0)
    
    df['3_last_increasing'] = list_3_last_increasing
    df['3_last_increasing_bool'] = list_3_last_increasing_bool
    
    list_4_last_increasing_sample = ('0123','1234','2345','3456','4567','5678','6789','0246','1357','2468','3579','0369')
    list_4_last_increasing = list()
    
    for x in list_4_last:
      check = 0
      for j in list_4_last_increasing_sample:
        if j in x:
            list_4_last_increasing.append(x)
            check = 1
            list_4_last_increasing_bool.append(1)
            break
        
      if check == 0:
        list_4_last_increasing.append("n") # 4-last increasing number 
        list_4_last_increasing_bool.append(0)
    
    df['4_last_increasing'] = list_4_last_increasing
    df['4_last_increasing_bool'] = list_4_last_increasing_bool
    
    list_5_last_increasing_sample = ('01234','12345','23456','34567','45678','56789','02468','13579')
    list_5_last_increasing = list()
    
    for x in list_5_last:
      check = 0
      for j in list_5_last_increasing_sample:
        if j in x:
            list_5_last_increasing.append(x)
            check =1
            list_5_last_increasing_bool.append(1)
            break
      if check == 0:      
        list_5_last_increasing.append("n") # 3-last increasing number 
        list_5_last_increasing_bool.append(0)
    
    df['5_last_increasing'] = list_5_last_increasing
    df['5_last_increasing_bool'] = list_5_last_increasing_bool
    
    return df

In [ ]:
df = last_special_case(df)
df.sample(10)

In [ ]:
def features_from_sum(df):
    list_number = df['phone_number'].to_list()
    list_sum = []
    for i in range(len(list_number)):
      tmp = 0
      for j in range(len(list_number[i])):
        tmp += int(list_number[i][j])
      list_sum.append(tmp)
    
    list_phong_thuy = []
    
    for i in range(len(list_sum)):
      list_phong_thuy.append(list_sum[i]%10)
    
    df['phong_thuy_1'] = list_phong_thuy
    
    list_phong_thuy_2 = []
    
    for i in range(len(list_sum)):
      k = list_sum[i]
      ans = 0
      while(k>=0):
        ans+=k%10
        k = math.floor(k/10)
        if (k==0 and ans >= 10):
          k = ans
          ans = 0
        if (k==0 and ans < 10):
          list_phong_thuy_2.append(ans)
          break
      
    df['phong_thuy_2'] = list_phong_thuy_2
    
    return df

In [ ]:
df = features_from_sum(df)
df.shape

In [ ]:
def creating_label(df):
    price = np.array(df['price_vnd'])
    label = []

    for i in range(df.shape[0]):
      if price[i] <= 1.6e6:
        label.append(0)
        continue
      if price[i] <= 3.6e6:
        label.append(1)
        continue
      if price[i] <= 6.5e6:
        label.append(2)
        continue
      if price[i] <= 10.8e6:
        label.append(3)
        continue
      if price[i] <= 1.5e7:
        label.append(4)
        continue
      if price[i] <= 6.1e7:
        label.append(5)
        continue
      if price[i] <= 1.5e8:
        label.append(6)
        continue
      if price[i] <= 3.1e8:
        label.append(7)
        continue
      if price[i] <= 5.3e8:
        label.append(8)
        continue
  
      label.append(9)

    df['label'] = label
    
    return df


In [ ]:
df = creating_label(df)
df.head(10)

In [ ]:
def creating_features_by_combine_pos(df):
    for i in range(1,9):
        for j in range(i+1,10):
            df['pos_'+str(i)+"_"+str(j)] = df['pos_' + str(i)] + df['pos_' + str(j)]
    
    for i in range(1,8):
        for j in range(i+1,9):
            for k in range(j+1,10):
                df['pos_'+str(i)+"_"+str(j)+"_"+str(k)] = df['pos_' + str(i)] + df['pos_' + str(j)] + df['pos_'+str(k)]
      
#     for i in range(1,7):
#         for j in range(i+1,8):
#             for k in range(j+1,9):
#                 for l in range(k+1,10):
#                     #df_tmp = pd.DataFrame()
#                     df['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)] = df['pos_' + str(i)] \
#                     + df['pos_' + str(j)] + df['pos_'+str(k)] + df['pos_'+str(l)]
#                     df_ans = pd.concat([df_ans, df_tmp],axis = 1)
                
#     df['pos_1_2_8_9'] = df['pos_1'] + df['pos_2'] + df['pos_8'] + df['pos_9']
    
#     for i in range(1,7):
#         df['pos_'+str(i)+"_"+str(i+1)+"_"+str(i+2)+"_"+str(i+3)] = df['pos_' + str(i)] + df['pos_' + str(i+1)] + df['pos_'+str(i+2)] + df['pos_' + str(i+3)]
    
    return df
    
# def creating_features_by_combine_pos(df):
#     df_ans = df
    
# #     for i in range(1,10):
# #         df['pos_' + str(i)] = df['pos_'+str(i)].astype(str)
        
#     for i in range(1,7):
#         for j in range(i+1,8):
#             for k in range(j+1,9):
#                 for l in range(k+1,10):
#                     df_tmp = pd.DataFrame()
#                     df_tmp['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)] = df['pos_' + str(i)] \
#                     + df['pos_' + str(j)] + df['pos_'+str(k)] + df['pos_'+str(l)]
#                     df_ans = pd.concat([df_ans, df_tmp],axis = 1)
    
    
#     return df_ans

In [ ]:
df = creating_features_by_combine_pos(df)
df.shape

In [ ]:
# df['pos_1_2_3_4'].value_counts()

### Tuan feats

In [ ]:
def sim_loc_phat(df):
    list_number = df['phone_number'].to_list()
    list_2_last = list() #extract 2 last digit

    list_4_last = list() #extract 3 last digit
    
    for i in range(len(list_number)):
        str_tmp_2_last = list_number[i][-2:]
        list_2_last.append(str_tmp_2_last)
    
        str_tmp_4_last = list_number[i][-4:]
        list_4_last.append(str_tmp_4_last)
        
        
    list_phat_loc_2_bool = []
    list_phat_loc_2_cat = []

    list_phat_loc_4_bool = []
    list_phat_loc_4_cat = []

    list_phat_loc_2_sample = ['68','86']
    list_phat_loc_4_sample = ['6688','6868','6886', '8668']

    for x in list_2_last:
        if list_phat_loc_2_sample.count(x)>0:
            list_phat_loc_2_bool.append(1)
            list_phat_loc_2_cat.append(x)
        else:
            list_phat_loc_2_bool.append(0)
            list_phat_loc_2_cat.append('')
        

    for x in list_4_last:
        if list_phat_loc_4_sample.count(x)>0:
            list_phat_loc_4_bool.append(1)
            list_phat_loc_4_cat.append(x)
        else:
            list_phat_loc_4_bool.append(0)
            list_phat_loc_4_cat.append('')      
        
    df['phat_loc_2_bool'] = list_phat_loc_2_bool
    df['phat_loc_2_cat'] = list_phat_loc_2_cat

    df['phat_loc_4_bool'] = list_phat_loc_4_bool
    df['phat_loc_4_cat'] = list_phat_loc_4_cat
    
    return df


In [ ]:
df = sim_loc_phat(df)

In [ ]:
df[df['phat_loc_2_bool'] == 1].shape

In [ ]:
def increasing_3_4(df):
    list_number = df['phone_number'].to_list()
    list_double_3_increasing = []
    
    list_3_last_increasing_bool = df['3_last_increasing_bool'].to_list()
    list_4_last_increasing_bool = df['4_last_increasing_bool'].to_list()
    for i in range(len(list_number)):
        if (list_3_last_increasing_bool[i] == 1):
            if (list_number[i][3:6] == list_number[i][6:9]):
                list_double_3_increasing.append(1)
            else: list_double_3_increasing.append(0)
        else: list_double_3_increasing.append(0)

    df['list_double_3_increasing']= list_double_3_increasing
    
    list_double_4_increasing = []

    for i in range(len(list_number)):
        if (list_4_last_increasing_bool[i] == 1):
            if (list_number[i][1:5] == list_number[i][5:9]):
                list_double_4_increasing.append(1)
            else: list_double_4_increasing.append(0)
        else: list_double_4_increasing.append(0)
    
    df['list_double_4_increasing']= list_double_4_increasing
    
    return df
        

In [ ]:
df = increasing_3_4(df)

In [ ]:
def double_3_last(df):
    list_number = df['phone_number'].to_list()
    list_double_3_last = []

    for i in range(len(list_number)):
        if (list_number[i][3:6] == list_number[i][6:9]):
            list_double_3_last.append(1)
        else: list_double_3_last.append(0)
    
    df['list_double_3_last']= list_double_3_last
    return df

In [ ]:
df = double_3_last(df)

In [ ]:
df[df['list_double_3_last']==1]

In [ ]:
def triple_2_last(df):
    list_number = df['phone_number'].to_list()
    list_triple_2_last_r1 = [] # list for triple 2 last but too not 'beautiful'
    tmp_list = [5,6,8,9]

    for i in range(len(list_number)):
        if(int(list_number[i][8]) in tmp_list and int(list_number[i][7]) in tmp_list ):
            list_triple_2_last_r1.append(0)
        else:
            if (list_number[i][3:5] == list_number[i][5:7] == list_number[i][7:9]):
                list_triple_2_last_r1.append(1)
            else:
                list_triple_2_last_r1.append(0)
                
    df['triple_2_last_r1'] = list_triple_2_last_r1
    return df

In [ ]:

df = triple_2_last(df)

In [ ]:
df[df['triple_2_last_r1'] == 1]

In [ ]:
# Sim kep duoi (ABB.CDD) va sim lap dau(ABC.ABD), lap cuoi (ABC.DBC)
def sim_6_last(df):
    list_number = df['phone_number'].to_list()
    list_kep_duoi_bool = []
    list_kep_duoi_cat = []

    list_lap_dau_bool = []
    list_lap_dau_cat = []

    list_lap_cuoi_bool = []
    list_lap_cuoi_cat = []

    for x in list_number:
        if (x[4:6] == x[7:]):
            list_lap_cuoi_bool.append(1)
            list_lap_cuoi_cat.append(x[4:6])
        
            if (x[7] == x[8]):
                list_kep_duoi_bool.append(1)
                list_kep_duoi_cat.append(x[8])
            else:
                list_kep_duoi_bool.append(0)
                list_kep_duoi_cat.append('')
        else:
            list_lap_cuoi_bool.append(0)
            list_lap_cuoi_cat.append('')
        
            list_kep_duoi_bool.append(0)
            list_kep_duoi_cat.append('')
        
        if (x[3:5] == x[6:8]):
            list_lap_dau_bool.append(1)
            list_lap_dau_cat.append(x[3:5])
        else:
            list_lap_dau_bool.append(0)
            list_lap_dau_cat.append('')
    df['kep_duoi_bool'] = list_kep_duoi_bool
    df['kep_duoi_cat'] = list_kep_duoi_cat

    df['lap_dau_bool'] = list_lap_dau_bool
    df['lap_dau_cat'] = list_lap_dau_cat

    df['lap_cuoi_bool'] = list_lap_cuoi_bool
    df['lap_cuoi_cat'] = list_lap_cuoi_cat
    
    return df

In [ ]:

df = sim_6_last(df)

In [ ]:
df[df['lap_dau_bool']==1]

In [ ]:
def special_4_sim(df):
    list_number = df['phone_number'].to_list()
    list_2_lap_t1 = [] #ABAB, A>B
    list_2_lap_t2 = [] # ABAB, A<B

    list_2_lap_t3 = [] # AABB, A>B
    list_2_lap_t4 = [] # AABB, A<B

    for i in range(len(list_number)):
        if (list_number[i][5:7] == list_number[i][7:]):
            if (int(list_number[i][7]) < int(list_number[i][8])):
                list_2_lap_t2.append(1)
                list_2_lap_t1.append(0)
            elif(int(list_number[i][7]) > int(list_number[i][8])):
                list_2_lap_t1.append(1)
                list_2_lap_t2.append(0)
            else:
                list_2_lap_t1.append(0)
                list_2_lap_t2.append(0)

        else:
            list_2_lap_t1.append(0)
            list_2_lap_t2.append(0)


        if (list_number[i][5]==list_number[i][6] and list_number[i][7]==list_number[i][8]):
            if(list_number[i][5] < list_number[i][7]):
                list_2_lap_t4.append(1)
                list_2_lap_t3.append(0)
            elif(list_number[i][5] > list_number[i][7]):
                list_2_lap_t4.append(0)
                list_2_lap_t3.append(1)
            else:
                list_2_lap_t4.append(0)
                list_2_lap_t3.append(0)

        else:
            list_2_lap_t4.append(0)
            list_2_lap_t3.append(0)
    df['2_lap_t1'] = list_2_lap_t1
    df['2_lap_t2'] = list_2_lap_t2
    df['2_lap_t3'] = list_2_lap_t3
    df['2_lap_t4'] = list_2_lap_t4
    
    return df

In [ ]:

df = special_4_sim(df)


In [ ]:
df[df['2_lap_t4']==1]

In [ ]:
def ong_dia(df):
    list_number = df['phone_number'].to_list()
    list_4_last = []
    for i in range(len(list_number)):
        str_tmp_4_last = list_number[i][-4:]
        list_4_last.append(str_tmp_4_last)
    
    list_lap_ong_dia = [] #7997, 8998
    list_lap_ong_dia_last = []
    list_lap_ong_dia_sample = ['7997', '8998']

    list_kep_ong_dia = [] #7799, 8899
    list_kep_ong_dia_last = []
    list_kep_ong_dia_sample = ['7799', '8899']

    list_dao_ong_dia_1 = [] #9779, 9889
    list_dao_ong_dia_1_last = []
    list_dao_ong_dia_1_sample = ['9779', '9889']

    list_dao_ong_dia_2 = [] #7997, 8998
    list_dao_ong_dia_2_last = []
    list_dao_ong_dia_2_sample = ['7997', '8998']

    for i in range(len(list_number)):
        #lap ong dia
        lap_ong_dia_score = 0
        lap_ong_dia_last_score = 0
        for tmp_str in list_lap_ong_dia_sample:
            if tmp_str in list_number[i]:
                lap_ong_dia_score = lap_ong_dia_score +1

            if (tmp_str == list_4_last[i]):
                lap_ong_dia_last_score = lap_ong_dia_last_score +1

        list_lap_ong_dia.append(lap_ong_dia_score)
        list_lap_ong_dia_last.append(lap_ong_dia_last_score)

        #kep ong dia
        kep_ong_dia_score = 0
        kep_ong_dia_last_score = 0
        for tmp_str in list_kep_ong_dia_sample:
            if tmp_str in list_number[i]:
                kep_ong_dia_score = kep_ong_dia_score +1

            if (tmp_str == list_4_last[i]):
                kep_ong_dia_last_score = kep_ong_dia_last_score +1

        list_kep_ong_dia.append(kep_ong_dia_score)
        list_kep_ong_dia_last.append(kep_ong_dia_last_score)   

        #dao ong dia 1
        dao_ong_dia_1_score = 0
        dao_ong_dia_1_last_score = 0
        for tmp_str in list_dao_ong_dia_1_sample:
            if tmp_str in list_number[i]:
                dao_ong_dia_1_score = dao_ong_dia_1_score +1

            if (tmp_str == list_4_last[i]):
                dao_ong_dia_1_last_score = dao_ong_dia_1_last_score +1

        list_dao_ong_dia_1.append(dao_ong_dia_1_score)
        list_dao_ong_dia_1_last.append(dao_ong_dia_1_last_score)   

        #dao ong dia 22
        dao_ong_dia_2_score = 0
        dao_ong_dia_2_last_score = 0
        for tmp_str in list_dao_ong_dia_2_sample:
            if tmp_str in list_number[i]:
                dao_ong_dia_2_score = dao_ong_dia_2_score +1

            if (tmp_str == list_4_last[i]):
                dao_ong_dia_2_last_score = dao_ong_dia_2_last_score +1

        list_dao_ong_dia_2.append(dao_ong_dia_2_score)
        list_dao_ong_dia_2_last.append(dao_ong_dia_2_last_score)  
    
    df['lap_ong_dia'] = list_lap_ong_dia 
    df['lap_ong_dia_last'] = list_lap_ong_dia_last 
    df['kep_ong_dia'] = list_kep_ong_dia 
    df['kep_ong_dia_last'] = list_kep_ong_dia_last 
    df['dao_ong_dia_1'] = list_dao_ong_dia_1
    df['dao_ong_dia_1_last'] = list_dao_ong_dia_1_last 
    df['dao_ong_dia_2'] = list_dao_ong_dia_2
    df['dao_ong_dia_2_last'] = list_dao_ong_dia_2_last
    
    return df


In [ ]:

df = ong_dia(df)

In [ ]:
df[df['dao_ong_dia_2_last']==1]

In [ ]:
def special_6_last(df):
    list_number = df['phone_number'].to_list()
    list_ABBBBA_1 = [] #A<B
    list_ABBBBA_2 = [] #A>B

    list_ABBA_1 = [] #A<B
    list_ABBA_2 = [] #A>B

    for i in range(len(list_number)):
        if (list_number[i][3] == list_number[i][8] and list_number[i][4] == list_number[i][5] and list_number[i][4] == list_number[i][6] and list_number[i][4] == list_number[i][7]):
            if (int(list_number[i][3]) < int(list_number[i][4])):
                list_ABBBBA_1.append(1)
                list_ABBBBA_2.append(0)
            elif(int(list_number[i][3]) > int(list_number[i][4])):
                list_ABBBBA_1.append(0)
                list_ABBBBA_2.append(1)
            else:
                list_ABBBBA_1.append(0)
                list_ABBBBA_2.append(0)
        else:
            list_ABBBBA_1.append(0)
            list_ABBBBA_2.append(0)

        if (list_number[i][5] == list_number[i][8] and list_number[i][6] == list_number[i][7]):
            if (int(list_number[i][5]) < int(list_number[i][6])):
                list_ABBA_1.append(1)
                list_ABBA_2.append(0)
            elif(int(list_number[i][5]) > int(list_number[i][6])):
                list_ABBA_1.append(0)
                list_ABBA_2.append(1)
            else:
                list_ABBA_1.append(0)
                list_ABBA_2.append(0)
        else:
            list_ABBA_1.append(0)
            list_ABBA_2.append(0)
    df['list_ABBBBA_1']= list_ABBBBA_1
    df['list_ABBBBA_2']= list_ABBBBA_2
    df['list_ABBA_1']= list_ABBA_1
    df['list_ABBA_2']= list_ABBA_2
    return df

In [ ]:

df = special_6_last(df)

In [ ]:
df[df['list_ABBA_2'] == 1]

In [ ]:
def check_easy_remember(df):
    list_number = df['phone_number'].to_list()
    list_1_3_5_iden_bool = []
    list_1_3_5_iden_cat = []

    list_2_4_6_iden_bool = []
    list_2_4_6_iden_cat = []

    list_3_5_7_iden_bool = []
    list_3_5_7_iden_cat = []

    list_4_6_8_iden_bool = []
    list_4_6_8_iden_cat = []

    list_5_7_9_iden_bool = []
    list_5_7_9_iden_cat = []

    for i in range(len(list_number)):
        if ( (list_number[i][0] == list_number[i][2]) and (list_number[i][0] == list_number[i][4])):
            list_1_3_5_iden_bool.append(1)
            list_1_3_5_iden_cat.append((int)(list_number[i][0]))
        else:
            list_1_3_5_iden_bool.append(0)
            list_1_3_5_iden_cat.append(-1)

        if ( (list_number[i][1] == list_number[i][3]) and (list_number[i][1] == list_number[i][5])):
            list_2_4_6_iden_bool.append(1)
            list_2_4_6_iden_cat.append((int)(list_number[i][1]))
        else:
            list_2_4_6_iden_bool.append(0)
            list_2_4_6_iden_cat.append(-1)

        if ((list_number[i][2] == list_number[i][4]) and (list_number[i][2] == list_number[i][6])):
            list_3_5_7_iden_bool.append(1)
            list_3_5_7_iden_cat.append((int)(list_number[i][2]))
        else:
            list_3_5_7_iden_bool.append(0)
            list_3_5_7_iden_cat.append(-1)

        if ((list_number[i][3] == list_number[i][5]) and (list_number[i][3] == list_number[i][7])):
            list_4_6_8_iden_bool.append(1)
            list_4_6_8_iden_cat.append((int)(list_number[i][3]))
        else:
            list_4_6_8_iden_bool.append(0)
            list_4_6_8_iden_cat.append(-1)

        if ( (list_number[i][4] == list_number[i][6]) and (list_number[i][4] == list_number[i][8])):
            list_5_7_9_iden_bool.append(1)
            list_5_7_9_iden_cat.append((int)(list_number[i][4]))
        else:
            list_5_7_9_iden_bool.append(0)
            list_5_7_9_iden_cat.append(-1)


    df['1_3_5_iden_bool'] = list_1_3_5_iden_bool
    df['1_3_5_iden_cat'] = list_1_3_5_iden_cat

    df['2_4_6_iden_bool'] = list_2_4_6_iden_bool
    df['2_4_6_iden_cat'] = list_2_4_6_iden_cat

    df['3_5_7_iden_bool'] = list_3_5_7_iden_bool
    df['3_5_7_iden_cat'] = list_3_5_7_iden_cat

    df['4_6_8_iden_bool'] = list_4_6_8_iden_bool
    df['4_6_8_iden_cat'] = list_4_6_8_iden_cat

    df['5_7_9_iden_bool'] = list_5_7_9_iden_bool
    df['5_7_9_iden_cat'] = list_5_7_9_iden_cat
    
    return df

In [ ]:
df = check_easy_remember(df)

In [ ]:
LIST_FEAT1 = ['pos_' + str(i) for i in range(1,10)]
LIST_FEAT2 = ['2_lucky','phong_thuy_1','phong_thuy_2','4_last','5_last',
              '4_last_iden', '5_last_iden' ,'4_last_increasing', 
              '5_last_increasing','2_iden','max_count','list_same','palindrome',
              'nam_sinh','3_last_iden','3_last_increasing']
LIST_FEAT3 = ['count_' + str(i) for i in range(0,10)]
LIST_FEAT4 = ['palindrome_length','list_same_substr_length','length']
LIST_FEAT5 = ['pos_' + str(i) + "_" +str(j) for i in range(1,9) for j in range(i+1,10)]
LIST_FEAT6 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k) for i in range(1,8) 
              for j in range(i+1,9) for k in range(j+1,10)]

LIST_FEAT9 = ['kep_duoi_cat','lap_dau_cat','lap_cuoi_cat','phat_loc_4_cat',
              'phat_loc_2_cat','5_combo_cat','6_combo_cat','4_combo_cat']
             
# ['3_combo_cat','4_combo_cat',]
#'kep_duoi_cat','lap_dau_cat','lap_cuoi_cat','phat_loc_4_cat','phat_loc_2_cat']
#             ,'1_3_5_iden_cat','2_4_6_iden_cat','3_5_7_iden_cat','4_6_8_iden_cat','5_7_9_iden_cat']
#              '

LIST_FEAT10 = ['list_ABBBBA_1','list_ABBBBA_2','list_ABBA_1','list_ABBA_2','lap_ong_dia','lap_ong_dia_last'
               ,'kep_ong_dia','kep_ong_dia_last','dao_ong_dia_1','dao_ong_dia_1_last','dao_ong_dia_2',
               'dao_ong_dia_2_last','2_lap_t1','2_lap_t2','2_lap_t3','2_lap_t4','kep_duoi_bool'
               ,'lap_dau_bool','lap_cuoi_bool','triple_2_last_r1','list_double_3_last','phat_loc_2_bool',
               'phat_loc_4_bool','nam_sinh_bool','bool_palindrome','bool_substr','2_lucky_bool','2_iden_bool','3_last_iden_bool',
                '4_last_iden_bool','5_last_iden_bool','3_last_increasing_bool','4_last_increasing_bool',
              '5_last_increasing_bool','1_3_5_iden_bool','2_4_6_iden_bool','3_5_7_iden_bool','4_6_8_iden_bool','5_7_9_iden_bool',
              '3_combo_bool','4_combo_bool','5_combo_bool','6_combo_bool','3_combo_s','3_combo_57','3_combo_689','4_combo_s',
               '4_combo_57','4_combo_689','list_double_3_increasing','list_double_4_increasing']

# LIST_FEAT7 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l) for i in range(1,7) 
#               for j in range(i+1,8) for k in range(j+1,9) for l in range(k+1,10)]

# LIST_FEAT7 = ['pos_'+str(i)+"_"+str(i+1)+"_"+str(i+2)+"_"+str(i+3) for i in range(1,7)]

# LIST_FEAT8 = ['pos_'+str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)+"_"+str(m) for i in range(1,6)
#               for j in range(i+1,7) for k in range(j+1,8) for l in range(k+1,9)
#               for m in range(l+1,10)]



class Categorical_encoder:
    def __init__(self):
        self.list_feat =LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT9
        self.list_encoder = []
    
    def check_containing_columns(self,list_columns):

        for i in self.list_feat:
            for j in range(len(list_columns)):
                if i == list_columns[j]:
                    break
                if j == len(list_columns) -1:
                    return False
        
        return True
                    
    def fit(self,df):
        
        if self.check_containing_columns(df.columns) is False:
               raise Exception("CAN NOT FIT BECAUSE DATAFRAME DO NOT HAVING ENOUGH REQUIRED COLUMNS")
        list_tmp = ['pos_' + str(i) for i in range(3,10)]
        list_tmp = list_tmp
        tmp = ""
        for i in self.list_feat:
            if (i in list_tmp):
                self.list_encoder.append(tmp)
                continue
            ordinal_encoder= preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                 unknown_value=-1)

            ordinal_encoder.fit(np.array(df[i]).reshape(-1,1))
            self.list_encoder.append(ordinal_encoder)
            tmp = ordinal_encoder

    
    def encode(self,df):
        list_tmp = ['pos_' + str(i) for i in range(3,10)]
        for i in range(len(self.list_feat)):
            #print(self.list_feat[i])
            df[self.list_feat[i]] = self.list_encoder[i].transform(np.array(df[self.list_feat[i]]).reshape(-1, 1)).flatten()
            #df[self.list_feat[i]] = self.list_encoder[i].transform(df[self.list_feat[i]])
        return df



In [ ]:
list_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT3 +  LIST_FEAT4 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT9 + LIST_FEAT10

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(df[list_features],df['label'], test_size=0.1, random_state=257)

In [ ]:
X_train.shape

In [ ]:
ct = Categorical_encoder()
ct.fit(df[list_features])
X_train = ct.encode(X_train)
X_test = ct.encode(X_test)


In [ ]:
del df
gc.collect()

In [ ]:
import xgboost as xgb

reg = xgb.XGBClassifier(n_estimators = 500, max_depth=20,tree_method="gpu_hist")

reg.fit(X_train, y_train)
y_train_predict = reg.predict(X_train)
print("XGB Accuracy Score of train set-> ",accuracy_score(y_train, y_train_predict)*100)

y_test_predict = reg.predict(X_test)
print(classification_report(y_test,y_test_predict))

In [ ]:
# rf = RandomForestClassifier(n_estimators = 200, max_features="sqrt")
# rf.fit(X_train,y_train)

# y_train_predict = rf.predict(X_train)
# print("RF Accuracy Score of train set-> ",accuracy_score(y_train, y_train_predict)*100)

# y_test_predict = rf.predict(X_test)
# print(classification_report(y_test,y_test_predict))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
fig, ax = plt.subplots(figsize=(40, 30))
plt.grid(False)
sns.set(font_scale=1.5)
plot_confusion_matrix(reg, X_test[list_features], y_test,cmap="OrRd",ax = ax)  
plt.show()


In [ ]:
plt.figure(figsize = (50,60))
plt.grid(False)

sorted_idx = reg.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], reg.feature_importances_[sorted_idx])
plt.xlabel("XGB Feature Importance")

In [ ]:
import pickle
pickle.dump(ct, open("encoder_xgb.pkl", "wb"))
pickle.dump(reg, open("xgb_reg_final.pkl", "wb"))
# pickle.dump(clf,open("ct.pkl", "wb"))

In [ ]:
# clf = CatBoostClassifier(task_type = "GPU", iterations = 1300,depth = 15)

# # ,iterations=1000,learning_rate = 0.15,depth =10)    
# # , ,cat_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT7 + LIST_FEAT8)
# clf.fit(X_train, y_train)
# y_train_predict = clf.predict(X_train)
# print("CBoost Accuracy Score of train set-> ",accuracy_score(y_train, y_train_predict)*100)

# y_test_predict = clf.predict(X_test)
# print(classification_report(y_test,y_test_predict))

In [ ]:
def guess_price(number, model,encoder):
    df = pd.DataFrame()
    df['phone_number'] = [number[1:]]
    df = removing_wrong_phone_number(df)
    df = creating_10_feats_from_single_pos(df)
    df = features_from_counting_numbers(df)
    df = sim_nam_sinh(df)
    df = symetric_4length_and_samesubtring_features(df)
    df = combo_feat(df)
    df = count_combo_features(df)
    df = features_2_inden_and_2_lucky_last(df)
    df = last_special_case(df)
    df = features_from_sum(df)
    df = creating_features_by_combine_pos(df)
    df = sim_loc_phat(df)
    df = double_3_last(df)
    df = triple_2_last(df)
    df = sim_6_last(df)
    df = special_4_sim(df)
    df = ong_dia(df)
    df = special_6_last(df)
    df = increasing_3_4(df)
    df = check_easy_remember(df)
    df = encoder.encode(df)
    list_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT3 + LIST_FEAT4 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT10 + LIST_FEAT9
    ans = model.predict(df[list_features])
 
    if(ans == 0):
        print("giá từ 0 đến 1,6tr vnd")
    elif ans == 1:
        print("giá từ 1,6 đến 3,6tr vnd")
    elif ans == 2:
        print("giá từ 3,6tr đến 6,5tr vnd")
    elif ans == 3:
        print("giá từ 6,5tr đến 10,8tr vnd")
    elif ans == 4:
        print("giá từ 10,8tr đến 15tr vnd")
    elif ans == 5:
        print("giá từ 15tr đến 61tr vnd")
    elif ans == 6:
        print("giá từ 61tr đến 150tr vnd")
    elif ans == 7:
        print("giá từ 150tr đến 310tr vnd")
    elif ans == 8:
        print("giá từ 310tr đến 530tr vnd")
    else:
        print("giá hơn 530tr")


In [ ]:
a = pickle.load(open("./xgb_reg_final.pkl", "rb"))
encoder = pickle.load(open("./encoder_xgb.pkl", "rb"))

In [ ]:
import time
start_time = time.time()
guess_price("0963126868",a,encoder)
print("run time is " + str(time.time() - start_time))

In [ ]:
# b = pickle.load(open("./cat.pkl", "rb"))
# encoder = pickle.load(open("./encoder.pkl", "rb"))